Desafio [Lighthouse] Desafio Ciência de Dados 2025-3. Neste notebook vamos ler um arquivo csv no Pandas e criar um modelo de regressão para prever o preço de um imóvel com base nos dados que foram fornecidos.
O notebook foi feito originalmente no Google Colab, recomendo este ambiente.

Antes de começar, inicie a instância do colab e faça upload do arquivo _teste_indicium_precificacao.csv_

Testei alguns modelos para resolver esse problema. Porém, o que apresentou o menor erro médio (RMSE e MAPE) foi o regressor lightgbm.

In [3]:
import pandas as pd
df = pd.read_csv('..\\data\\teste_indicium_precificacao.csv')
df.head()

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


Com nosso dataframe devidamente lido, podemos seguir importando as bibliotecas necessárias e configurando as features que usaremos em nosso modelo.

Nem todas as colunas foram utilizadas como features pois algumas não tinham correlação em termos de negócio (como o número de avaliações que não diretamente impactam o preço, e sim o oposto) e também apresentavam um coeficiente de correlação de Pearson fraco (o que é o caso para algumas features usadas, mas que haviam uma relação indireta como visto na análise exploratória, como minimo_noites, que conforme o minimo_noites aumenta, a tendência era o valor diminuir.)

In [4]:
#Aqui importamos as bibliotecas necessárias
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
#from sklearn.ensemble import RandomForestRegressor Comentado pois foi o teste inicial e não está sendo mais usado

#Usei esse valor aqui pois fica bem no centro de nova Iorque (Times Square)
centro = (40.7580, -73.9855)

#Definimos as Features
features = df[["latitude", "longitude", "bairro", "room_type", "bairro_group", "minimo_noites"]]
target = df["price"]

#Remoção de Outliers
#Essa remoção de Outliers foi agressiva, mas apresentou ótimos resultados em termos de precisão do modelo.
lower_bound = target.quantile(0.20)
upper_bound = target.quantile(0.80)

mask = (target >= lower_bound) & (target <= upper_bound)
features = features[mask]
target = target[mask]

#Aqui adicionamos a coluna distancia centro que é um dado adicional útil para o modelo
features["distancia_centro"] = features.apply(lambda row: geodesic((row["latitude"], row["longitude"]), centro).km, axis=1)

# Remove valores nulos
features = features.dropna()
target = target.loc[features.index]

# Separando as variáveis numéricas e categóricas
num_features = ["latitude", "longitude", "distancia_centro", "minimo_noites"]
cat_features = ["bairro_group", "room_type"]

#Aqui fazemos escalonamento nas variáveis numéricas para tornalas comparáveis
scaler = StandardScaler()
features[num_features] = scaler.fit_transform(features[num_features])

# Encodamos as variáveis categóricas com One-Hot Encoding para que o modelo possa usufruir
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
features_encoded = encoder.fit_transform(features[cat_features])

# Agora temos um dataframe final que combina todas as features
features_final = pd.concat(
    [
        features[num_features].reset_index(drop=True),
        pd.DataFrame(features_encoded, columns=encoder.get_feature_names_out()),
    ],
    axis=1,
)

# Dividimos em treino e teste (90% treino, 10% teste)
X_train, X_test, y_train, y_test = train_test_split(features_final, target, test_size=0.1, random_state=42)

In [5]:
target.mean()

np.float64(116.82084312803636)

Aqui aplicamos o LIGHTGBM Regressor para treinar nosso modelo

In [6]:
from lightgbm import LGBMRegressor

#Encontrei um ponto ótimo em 5000 estimators e uma learning_rate de 0.005
model = LGBMRegressor(n_estimators=5000, learning_rate=0.005, max_depth=20, random_state=42)
model.fit(X_train, y_train)

# Fazemos previsões e calculamos o erro
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

rmse, mape

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 845
[LightGBM] [Info] Number of data points in the train set: 28714, number of used features: 12
[LightGBM] [Info] Start training from score 116.873233


(np.float64(31.268504921322968), 0.22339495175701424)

Com o modelo pronto podemos fazer a predição do objeto passado no e-mail

In [7]:
# Criar DataFrame para a nova predição
dados_teste = pd.DataFrame([{
    "latitude": 40.75362,
    "longitude": -73.98377,
    "bairro_group": "Manhattan",
    "room_type": "Entire home/apt",
    "minimo_noites": 1,
    "numero_de_reviews": 45,
    "disponibilidade_365": 355,
    "calculado_host_listings_count": 2
}])

# Precisamos calcular a distância até o centro e adicionar ao dataframe
centro = (40.7580, -73.9855)
dados_teste["distancia_centro"] = geodesic((dados_teste["latitude"].values[0], dados_teste["longitude"].values[0]), centro).km

# Além disso precisamos aplicar o scaler também e o one-hot-encoding
num_features = ["latitude", "longitude", "distancia_centro", "minimo_noites"]
dados_teste[num_features] = scaler.transform(dados_teste[num_features])  # Usar o scaler treinado
dados_categ_encoded = encoder.transform(dados_teste[["bairro_group", "room_type"]])
dados_teste_final = pd.concat(
    [
        dados_teste[num_features].reset_index(drop=True),
        pd.DataFrame(dados_categ_encoded, columns=encoder.get_feature_names_out()),
    ],
    axis=1,
)

#Com o dataframe pronto basta passar para o modelo prever
preco_previsto = model.predict(dados_teste_final)

print(f"Preço previsto: ${preco_previsto[0]:.2f}")


Preço previsto: $154.80


Ótimo, o preço previsto foi de $154.80 agora podemos exportar nosso modelo usando pickle

In [8]:
import pickle

with open("modelo_lightgbm.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

with open("scaler.pkl", "wb") as scaler_file:
    pickle.dump(scaler, scaler_file)

with open("encoder.pkl", "wb") as encoder_file:
    pickle.dump(encoder, encoder_file)

print("Modelo, scaler e encoder salvos")


Modelo, scaler e encoder salvos


Com tudo isso pronto é só baixar os arquivos .pkl 